## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-08-55-09 +0000,bbc,Fourteen killed in Russian drone and missile a...,https://www.bbc.com/news/articles/cvg3y7m2gz0o...
1,2025-08-28-08-40-08 +0000,nyt,What to Know About Jimmy Lai’s Trial in Hong Kong,https://www.nytimes.com/2025/08/27/world/asia/...
2,2025-08-28-08-34-26 +0000,bbc,Caning and fines for those caught in Singapore...,https://www.bbc.com/news/articles/cz60z98nzx7o...
3,2025-08-28-08-34-04 +0000,wapo,What we know about the Minneapolis Catholic sc...,https://www.washingtonpost.com/nation/2025/08/...
4,2025-08-28-08-32-02 +0000,nyt,Jimmy Lai’s Freedom May Now Hinge on Beijing a...,https://www.nytimes.com/2025/08/27/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2300/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,56
23,school,14
141,new,14
21,minneapolis,14
24,shooting,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
81,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,148
50,2025-08-28-03-32-00 +0000,wsj,President Trump on Wednesday fired Robert Prim...,https://www.wsj.com/politics/policy/trump-fire...,107
227,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...,106
271,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...,101
16,2025-08-28-07-52-48 +0000,wapo,Dozen killed in Kyiv after fiercest Russian at...,https://www.washingtonpost.com/world/2025/08/2...,100


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
81,148,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...
51,73,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
10,63,2025-08-28-08-06-34 +0000,bbc,FBI investigates Minneapolis school shooting a...,https://www.bbc.com/news/articles/c7760x87r4po...
149,47,2025-08-27-21-29-33 +0000,nypost,Zohran Mamdani would easily win NYC mayor race...,https://nypost.com/2025/08/27/us-news/zohran-m...
201,43,2025-08-27-17-56-38 +0000,latimes,"Cal Fire captain fatally shot his girlfriend, ...",https://www.latimes.com/california/story/2025-...
227,42,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...
29,40,2025-08-28-05-18-29 +0000,nyt,"North Korea’s Leader, Kim Jong-un, to Attend M...",https://www.nytimes.com/2025/08/28/world/asia/...
299,40,2025-08-27-11-15-50 +0000,cbc,Denmark summons top U.S. diplomat after report...,https://www.cbc.ca/news/world/denmark-us-green...
139,40,2025-08-27-21-38-57 +0000,nypost,CDC Director Susan Monarez fired after refusin...,https://nypost.com/2025/08/27/us-news/cdc-dire...
318,37,2025-08-27-10-00-00 +0000,nypost,Grassley insists to ‘Pod Force One’ that Biden...,https://nypost.com/2025/08/27/us-news/grassley...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
